In [3]:
print("coucou")

coucou

In [4]:
using Polyhedra
using JuMP
using LinearAlgebra
using Gurobi #, Cbc, GLPK
using Plots
using JLD
using Combinatorics

In [5]:
include("../lib/linear_system_v3.jl")
print("Done")

Done

In [6]:
include("../lib/fixed_height_planar_lipm.jl")
print("Done")

Done

In [7]:
include("../lib/utils_MILP.jl");
print("Done")

Done

In [8]:
# data of the problem
enforceRegular=false
minimizeScaling=false #enforceRegular

tested_T=22:2:30

22:2:30

In [9]:
len=length(tested_T)

instantiation_times=zeros(len)
optimization_times=zeros(len)
feasibilities=zeros(len)
numberSchedulings=zeros(len)


for i=1:len
    # data of the problem
    T=tested_T[i]
    N_m=round(Int8,T/2)
    N_c=N_m
    
    println("\n====== T=$T ======\n")
    
    # Instantiation (including loading the problem)
    time_beginning=time_ns()
    
    system, _ = get_lipm(T)
    model,C_bar,S_bar, P_z_all,z_tilde, P_u_all, u_tilde, polEta=instantiateASAP(system,N_m,N_c,T; minimizeScaling=minimizeScaling,enforceRegular=enforceRegular);
    
    time_ending=time_ns()
    instantiation_time=(time_ending-time_beginning)/1e9
    print("\n==== Instantiation time = $(instantiation_time) seconds ====\n")
    instantiation_times[i]=instantiation_time
    
    # Set an optimozer and solve the problem
    set_optimizer(model, Gurobi.Optimizer)
    
    # Solving the problem
    time_beginning=time_ns()
    
    optimize!(model)
    
    time_ending=time_ns()
    optimization_time=(time_ending-time_beginning)/1e9
    print("\n\n==== Optimization time = $(optimization_time) seconds ====\n\n")
    optimization_times[i]=optimization_time
    
    feasibilities[i] = (termination_status(model) == MOI.TerminationStatusCode(1)) # 1 if feasible
    
    numberSchedulings[i]=length(combinations(0:T-1,N_m))*length(combinations(0:T-1,N_c))
end


====== T=22 ======

Estimated error in calculating first entry of discretized B: 6.938893903907228e-18
Estimated error in calculating second entry of discretized B: 0.0

==== Instantiation time = 60.026531992 seconds ====
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 41236 rows, 25514 columns and 1153396 nonzeros
Model fingerprint: 0xa80d2cbb
Variable types: 25470 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+06]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 7e+02]
Presolve removed 33006 rows and 12886 columns
Presolve time: 0.89s
Presolved: 8230 rows, 12628 columns, 248862 nonzeros
Variable types: 12584 continuous, 44 integer (44 binary)

Root relaxation: objective 0.000000e+00, 5809 iterations, 3.13 seconds
Total elapsed time = 15.09s
Total elapsed time = 36.96s

    Nodes    |    Current Node    |     Objective Bou


====== T=30 ======

Estimated error in calculating first entry of discretized B: 6.938893903907228e-18
Estimated error in calculating second entry of discretized B: 0.0

==== Instantiation time = 3.772666991 seconds ====
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 75660 rows, 46786 columns and 3726796 nonzeros
Model fingerprint: 0x76723447
Variable types: 46726 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+07]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 8e+03]
Presolve removed 60598 rows and 23566 columns
Presolve time: 3.39s
Presolved: 15062 rows, 23220 columns, 750078 nonzeros
Variable types: 23160 continuous, 60 integer (60 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   27

In [10]:
# save results
io = open("results/horizonScalingAnalysis.txt", "w") do io
    println(io,"T, instantiation_time [sec], optimization_times [sec], numberSchedulings, feasibility")
    for i=1:len
        print(io, tested_T[i])
        print(io,", ")
        print(io, instantiation_times[i])
        print(io,", ")
        print(io, optimization_times[i])
        print(io,", ")
        print(io, numberSchedulings[i])
        print(io,", ")
        print(io, feasibilities[i])
        println(io)
    end
end

In [11]:
instantiation_times

5-element Array{Float64,1}:
 60.026531992
  1.402219308
  2.494974213
  2.839054634
  3.772666991

In [12]:
optimization_times

5-element Array{Float64,1}:
   98.710842218
  393.665445925
  351.247031344
 2037.170254811
 3284.713340124

In [13]:
numberSchedulings

5-element Array{Float64,1}:
 4.97634306624e11
 7.312459672336e12
 1.0817248036e14
 1.60934159556e15
 2.40614450109504e16

In [14]:
feasibilities

5-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0